In [2]:
from gensim.corpora import Dictionary

In [3]:
import numpy as np
from gensim import matutils
from gensim.models.ldamodel import LdaModel
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [4]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from gensim.sklearn_api import TfIdfTransformer

In [5]:
rand = np.random.mtrand.RandomState(1) # set seed for getting same result
cats=['sci.space', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=cats, shuffle=True)

In [6]:
data_texts = [_.split() for _ in newsgroups.data]
id2word = Dictionary(data_texts)
corpus = [id2word.doc2bow(i.split()) for i in newsgroups.data]

In [7]:
%%script false 
print (len(corpus))

1177


In [7]:
tfidf_model = TfIdfTransformer()
tfidf_model.fit(corpus)

clf = linear_model.LogisticRegression(penalty='l2', C=0.1)
text_tfidf = Pipeline((('tfidf_model', tfidf_model), ('classifier', clf)))

text_tfidf.fit(corpus, newsgroups.target)
score = text_tfidf.score(corpus, newsgroups.target)
print(score)

ValueError: setting an array element with a sequence.

In [ ]:
tfidf_model = TfIdfTransformer(pivot_norm=True ,slope=1 )
tfidf_model.fit(corpus)

lda_model = LdaTransformer(num_topics=2, passes=10, minimum_probability=0, random_state=np.random.seed(0))
clf = linear_model.LogisticRegression(penalty='l2', C=0.1)
text_tfidf = Pipeline((('tfidf_model', tfidf_model), ('ldamodel', lda_model), ('classifier', clf)))

text_tfidf.fit(corpus, data.target)
score = text_tfidf.score(corpus, data.target)
print(score)

In [17]:
tfidf = TfIdfTransformer()
X_train_tfidf = tfidf.fit(corpus)
print (X_train_tfidf)
model = LogisticRegression()
model = model.fit(X_train_tfidf, newsgroups.target)

# check the accuracy on the training set
model.score(corpus, newsgroups.target)

TfIdfTransformer(dictionary=None, id2word=None, normalize=True, pivot=None,
         pivot_norm=False, slope=0.65, smartirs='ntc',
         wglobal=<function df2idf at 0x7f3e216df7d0>,
         wlocal=<function identity at 0x7f3e274c1d70>)


TypeError: float() argument must be a string or a number

In [11]:
np.array(corpus)

array([list([(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 3), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 4), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 3), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1)]),
       list([(9, 5), (13, 1), (19, 1), (20, 1), (24, 1), (26, 1), (30, 2), (37, 8), (39, 1), (43, 1), (44, 1), (49, 2), (59, 2), (60, 3), (65, 1), (68, 8), (75, 1), (77, 15), (81, 6), (

In [18]:
x = []
for doc in corpus:
    x.append(X_train_tfidf.transform(doc))

In [21]:

model = LogisticRegression()
model = model.fit(x, newsgroups.target)


ValueError: setting an array element with a sequence.